In [1]:
# https://blog.langchain.dev/reflection-agents/

# https://github.com/langchain-ai/langgraph/blob/main/examples/reflection/reflection.ipynb?ref=blog.langchain.dev

## Generation LLM

Here, we set up the generation llm 

In [2]:
from pathlib import Path
import sys
sys.path.append(
    str(Path.cwd().parents[1])
);

import sys

from langchain_community.document_loaders import DirectoryLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.llms import Ollama
from utilities.ollama import verify_ollama_model_present

MODEL_NAME = "openchat"
# https://ollama.com/library/openchat

llm = Ollama(model=MODEL_NAME)
embeddings = OllamaEmbeddings(model=MODEL_NAME)


# make sure that the ollama is available to use locally (might need to ollama pull)
verify_ollama_model_present("openchat")

validating model exists...


In [3]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.messages import AIMessage, BaseMessage, HumanMessage

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are an essay assistant tasked with writing excellent 5-paragraph essays.
             Generate the best essay possible for the user's request.
             If the user provides critique, respond with a revised version of your previous attempts.""",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

generate = prompt | llm

In [4]:
essay = ""
request = HumanMessage(
    content="Write an essay on why the little prince is relevant in modern childhood"
)
for chunk in generate.stream({"messages": [request]}):
    print(chunk, end='')
    essay += chunk

 Title: The Relevance of The Little Prince in Modern Childhood

Introduction

Antoine de Saint-Exupéry's "The Little Prince" has captured the hearts and minds of readers for generations. Although it was first published in 1943, the themes and messages found within this allegorical novella continue to resonate with modern childhood experiences. This essay will discuss how the story of The Little Prince remains relevant to today's children by exploring its themes of friendship, self-discovery, and the importance of maintaining a sense of wonder in an increasingly complex world.

Body Paragraph 1: Friendship and Loneliness

One central theme of "The Little Prince" is the significance of friendship and the dangers of loneliness. The Little Prince's journey through various planets teaches him about different relationships, ultimately leading to his connection with the narrator, a pilot stranded in the desert. This bond becomes crucial for both characters, as they find solace and companionsh

## Reflection LLM

Next, we set up the reflection LLM

In [6]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """You are grading an essay submission. Generate critique and recommendations for the user's submission. 
            Provide detailed recommendations, including requests for length, depth, style, etc.
            Use this template in your answer:
            Grade: A to F
            Areas of improvement (if any): """,
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
reflect = reflection_prompt | llm

In [7]:
reflection = ""
for chunk in reflect.stream({"messages": [request, HumanMessage(content=essay)]}):
    print(chunk, end="")
    reflection += chunk

 Grade: A

Areas of improvement (if any): 
The essay is well-structured, with a clear introduction, body paragraphs, and conclusion. It effectively addresses the themes of friendship, self-discovery, and maintaining a sense of wonder in "The Little Prince" and how these themes are relevant to modern childhood experiences. The language used is clear and concise, making it easy to follow and understand. However, there could be more specific examples from the text to support the points made, as well as some additional exploration of how these themes can be applied to today's children's lives. So the overall score is A.

## Rerun generation model with feedback

In [28]:
for chunk in generate.stream(
    {"messages": [request, AIMessage(
        content=essay), HumanMessage(content=reflection)]}
):
    print(chunk, end="")

 Title: The Relevance of The Little Prince in Modern Childhood

Introduction:
Antoine de Saint-Exupéry's "The Little Prince," a captivating tale of a young prince who traverses the universe, has enchanted generations of readers with its profound lessons about life and humanity. Although initially published in 1943, this timeless story remains highly relevant to modern childhood, as it delves into themes such as loneliness, the importance of meaningful relationships, and the necessity for a balance between imagination and reality.

Body:
I. The Power of Imagination: In today's world, children often face an overwhelming influx of information and stimulation, which can lead to stress and a lack of creativity. "The Little Prince" inspires children to embrace their imaginations by exploring the beauty of nature and the depth of human emotions. This narrative reminds young readers that imagination is crucial for personal growth and self-discovery, fostering empathy and understanding in an ev

## Wire up this sequence in a graph

In [10]:
from typing import List, Sequence

from langgraph.graph import END, MessageGraph


async def generation_node(state: Sequence[BaseMessage]):
    content = await generate.ainvoke({"messages": state})
    return AIMessage(content=content)


async def reflection_node(messages: Sequence[BaseMessage]):
    # Other messages we need to adjust
    cls_map = {"ai": HumanMessage, "human": AIMessage}
    
    # First message is the original user request. 
    translated = [messages[0]] + [
        cls_map[msg.type](content=msg.content) for msg in messages[1:]
    ]
    
    res = await reflect.ainvoke({"messages": translated})
    
    # We treat the output of this as human feedback for the generator
    return HumanMessage(content=res)


builder = MessageGraph()
builder.add_node("generate", generation_node)
builder.add_node("reflect", reflection_node)
builder.set_entry_point("generate")


def should_continue(state: List[BaseMessage]):
    """currently makes the graph execute until the state is at least 2 messages"""
    if len(state) > 3:
        return END
    return "reflect"

builder.add_conditional_edges("generate", should_continue)
builder.add_edge("reflect", "generate")
graph = builder.compile()

Small note: There are some minor changes when compared to the reference notebook
We had to make some changes in the code above because Langchain Runnables (the prompt | llm object) seems to be expected to return Messages but now instead return str.

In [11]:
async for event in graph.astream(
    [
        HumanMessage(
            content="Create a write up about the challenges of migrating from on-prem to cloud."
        )
    ],
):
    print(event)
    print("---\n")

{'generate': AIMessage(content=" Title: The Challenges of Migrating from On-Premises to Cloud Computing\n\nIntroduction:\nThe transition from on-premises infrastructure to cloud computing has become increasingly popular among businesses in recent years. While the potential benefits, such as cost savings and scalability, are well-known, migrating from an on-premises environment to a cloud-based solution is not without its challenges. This essay will explore some of the most significant obstacles organizations face when undertaking this migration process.\n\nParagraph 1: Security Concerns\nOne of the primary concerns for organizations considering a shift to cloud computing is security. Maintaining data privacy and ensuring compliance with various regulations can be challenging in a cloud environment. Companies must carefully assess how their data will be stored, processed, and managed by the cloud provider. Additionally, they need to evaluate the provider's security measures, such as enc

In [12]:
ChatPromptTemplate.from_messages(event[END]).pretty_print()

================================ Human Message =================================

Create a write up about the challenges of migrating from on-prem to cloud.

================================== Ai Message ==================================

 Title: The Challenges of Migrating from On-Premises to Cloud Computing

Introduction:
The transition from on-premises infrastructure to cloud computing has become increasingly popular among businesses in recent years. While the potential benefits, such as cost savings and scalability, are well-known, migrating from an on-premises environment to a cloud-based solution is not without its challenges. This essay will explore some of the most significant obstacles organizations face when undertaking this migration process.

Paragraph 1: Security Concerns
One of the primary concerns for organizations considering a shift to cloud computing is security. Maintaining data privacy and ensuring compliance with various regulations can be challenging in a cloud e